# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
city_df = pd.read_csv("Output/cities.csv")
city_df

,City,Latitude,Longitude,Temperature (F),Maximum Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Data
0,Hilo,19.7297,-155.0900,73.69,79.29,94,90,4.61,US,1631786705
1,Tasiilaq,65.6145,-37.6368,37.56,37.56,80,96,6.06,GL,1631786707
2,Mataura,-46.1927,168.8643,39.47,39.47,91,54,2.77,NZ,1631786709
3,Chui,-33.6971,-53.4616,46.56,46.56,83,0,6.51,UY,1631786709
4,Lucapa,-8.4192,20.7447,86.04,86.04,40,93,5.39,AO,1631786711
...,...,...,...,...,...,...,...,...,...,...
552,Kasempa,-13.4584,25.8338,95.20,95.20,14,63,4.88,ZM,1631787360
553,Hamilton,39.1834,-84.5333,62.58,67.05,95,90,4.61,US,1631787362
554,Prince Rupert,54.3161,-130.3201,50.29,53.29,93,90,4.61,CA,1631787363
555,Santa Rosa,14.3122,121.1114,82.94,84.11,78,99,5.59,PH,1631787077


In [18]:
#MAX HUMIDITY
max_humid = city_df["Humidity"].max()
max_humid

100

In [19]:
#Drop any rows will null values.
country_null = city_df.loc[city_df["Country"].isnull() == True,:]
nullcountry_index = country_null.index
nullcountry_index


Int64Index([125, 154, 338, 508], dtype='int64')

In [20]:
city_df.drop(city_df.index[nullcountry_index], axis=0, inplace=True)
city_df

,City,Latitude,Longitude,Temperature (F),Maximum Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Data
0,Hilo,19.7297,-155.0900,73.69,79.29,94,90,4.61,US,1631786705
1,Tasiilaq,65.6145,-37.6368,37.56,37.56,80,96,6.06,GL,1631786707
2,Mataura,-46.1927,168.8643,39.47,39.47,91,54,2.77,NZ,1631786709
3,Chui,-33.6971,-53.4616,46.56,46.56,83,0,6.51,UY,1631786709
4,Lucapa,-8.4192,20.7447,86.04,86.04,40,93,5.39,AO,1631786711
...,...,...,...,...,...,...,...,...,...,...
552,Kasempa,-13.4584,25.8338,95.20,95.20,14,63,4.88,ZM,1631787360
553,Hamilton,39.1834,-84.5333,62.58,67.05,95,90,4.61,US,1631787362
554,Prince Rupert,54.3161,-130.3201,50.29,53.29,93,90,4.61,CA,1631787363
555,Santa Rosa,14.3122,121.1114,82.94,84.11,78,99,5.59,PH,1631787077


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
#configure GMAPs
gmaps.configure(api_key=g_key)

In [22]:
#select the required datas only from the DATAFRAME above
lat_lng = city_df[["Latitude","Longitude"]]
weight = city_df["Humidity"]



In [23]:
#customised the size of the figure
figure_layout = {'width':'800px', 'height':'600px', 'border':'1px solid black', 'padding':'5px', 'margin':'0 auto 0 auto'}

#plot Heatmap
fig = gmaps.figure(layout = figure_layout,zoom_level=2,center=(0,0))

In [24]:
#create heat layer
heatlayer = gmaps.heatmap_layer(lat_lng,weights = weight, dissipating = False, max_intensity =100, point_radius=1)

In [25]:
#add layer
fig.add_layer(heatlayer)

#display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='5px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
# narrow down by selecting only the columns as my vacation requirement
# #already dropped null values in the previous section -- current city_df is a cleaned df
# max_temp = city_df["Maximum Temperature (F)"]
# cloud = city_df["Cloudiness"]
# wind_mph = city_df["Wind Speed"]


In [27]:
#CHANGING the format
#df["2nd"] = pd.to_numeric(df["2nd"])
city_df["Cloudiness"] = pd.to_numeric(city_df["Cloudiness"]).astype("float")


In [28]:
#Create a new Dataframe that fits in the 3 requirements
#max temperature lower than 80 degrees but higher than 70
#Wind Speed less than 10mph
#Zero cloudiness
#Drop any rows that don't contain all three conditions. You want to be sure that the weather is ideaL

ideal_vacation = city_df.loc[(city_df["Maximum Temperature (F)"]<80) & (city_df["Maximum Temperature (F)"]>70) & (city_df["Wind Speed"] <10) & (city_df["Cloudiness"] == 0)]
ideal_vacation

,City,Latitude,Longitude,Temperature (F),Maximum Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Data
42,Archidona,37.0965,-4.3887,71.15,74.23,53,0.0,3.22,ES,1631786757
183,Muros,42.7762,-9.0603,68.56,74.23,74,0.0,6.31,ES,1631786933
209,Bom Jesus,-9.0744,-44.3586,77.58,77.58,41,0.0,2.06,BR,1631786964
221,Diamantino,-14.4086,-56.4461,74.84,74.84,71,0.0,3.18,BR,1631786978
295,Cayenne,4.9333,-52.3333,78.10,78.10,89,0.0,2.30,GF,1631787069
380,Saint-Pierre,-21.3393,55.4781,77.56,79.18,64,0.0,8.05,RE,1631787165
398,Chaohu,31.6000,117.8667,76.96,76.96,61,0.0,6.13,CN,1631787188
423,Yumen,40.2833,97.2000,78.66,78.66,17,0.0,9.66,CN,1631787216
502,Palmas,-10.2128,-48.3603,76.87,76.87,69,0.0,3.44,BR,1631787311
522,Correntina,-13.3433,-44.6367,73.17,73.17,39,0.0,3.89,BR,1631787333


In [29]:
#check if there's any NA value // .dropna(how= "all")
ideal_vacation.isna == True

False

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
#SELECT only required columns from the ideal city above 
hotel_df = ideal_vacation[["City","Country","Latitude","Longitude"]]
hotel_df.head()

,City,Country,Latitude,Longitude
42,Archidona,ES,37.0965,-4.3887
183,Muros,ES,42.7762,-9.0603
209,Bom Jesus,BR,-9.0744,-44.3586
221,Diamantino,BR,-14.4086,-56.4461
295,Cayenne,GF,4.9333,-52.3333


In [31]:
#add a new column called HOTEL NAME
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-31-794c290d8374>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Latitude,Longitude,Hotel Name
42,Archidona,ES,37.0965,-4.3887,
183,Muros,ES,42.7762,-9.0603,
209,Bom Jesus,BR,-9.0744,-44.3586,
221,Diamantino,BR,-14.4086,-56.4461,
295,Cayenne,GF,4.9333,-52.3333,
380,Saint-Pierre,RE,-21.3393,55.4781,
398,Chaohu,CN,31.6000,117.8667,
423,Yumen,CN,40.2833,97.2000,
502,Palmas,BR,-10.2128,-48.3603,
522,Correntina,BR,-13.3433,-44.6367,


In [32]:
#use iterrows to iterate through pandas dataframe
for index,row in hotel_df.iterrows():
    
    #get the lat and long from the hotel_df (which is generated from ideal_vacation_df)
    city_lat= row["Latitude"]
    city_lng= row["Longitude"]
    city_name=row["City"]
##Set parameters to search for hotels with 5000 meters.
    location = f'{city_lat},{city_lng}'

    params = {"location":location, "radius": 5000 ,"types" : "accomodation", "keyword": "hotel","key":g_key}
    
    #Hit the Google Places API for each city's coordinates.
    print(f'Retrieving results for Index {index}: City {city_name}')
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    
    try:
    #EXTRACT RESULTS
    #pprint(response["results"][1]["name"])
        print(f'First hotel in {city_name} is {response["results"][0]["name"]}')
        hotel_df.loc[index,"Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result ... Skipping")
    print("----------------------")

print("------------End of Search--------------") 

Retrieving results for Index 42: City Archidona
First hotel in Archidona is Hotel Escua
----------------------
Retrieving results for Index 183: City Muros


C:\Users\linee\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


First hotel in Muros is Hotel do Porto
----------------------
Retrieving results for Index 209: City Bom Jesus
First hotel in Bom Jesus is Gurguéia Palace Hotel
----------------------
Retrieving results for Index 221: City Diamantino
First hotel in Diamantino is Kayabi Palace Hotel, Cervejaria e Restaurante
----------------------
Retrieving results for Index 295: City Cayenne
First hotel in Cayenne is Mercure Cayenne Royal Amazonia
----------------------
Retrieving results for Index 380: City Saint-Pierre
First hotel in Saint-Pierre is Le Battant Des Lames
----------------------
Retrieving results for Index 398: City Chaohu
First hotel in Chaohu is Chaohu Country Garden Phoenix Hotel
----------------------
Retrieving results for Index 423: City Yumen
Missing field/result ... Skipping
----------------------
Retrieving results for Index 502: City Palmas
First hotel in Palmas is ibis Palmas
----------------------
Retrieving results for Index 522: City Correntina
First hotel in Correntina 

In [33]:
hotel_missing = hotel_df.loc[hotel_df["Hotel Name"] == ""]
print(f'This index in dataframe {hotel_missing.index} is missing.')

This index in dataframe Int64Index([423], dtype='int64') is missing.


In [34]:
hotel_new_df = hotel_df.copy()
hotel_new_df = hotel_new_df.replace("", float('NaN'), regex=True)
hotel_df2 = hotel_new_df.dropna()
hotel_df2

,City,Country,Latitude,Longitude,Hotel Name
42,Archidona,ES,37.0965,-4.3887,Hotel Escua
183,Muros,ES,42.7762,-9.0603,Hotel do Porto
209,Bom Jesus,BR,-9.0744,-44.3586,Gurguéia Palace Hotel
221,Diamantino,BR,-14.4086,-56.4461,"Kayabi Palace Hotel, Cervejaria e Restaurante"
295,Cayenne,GF,4.9333,-52.3333,Mercure Cayenne Royal Amazonia
380,Saint-Pierre,RE,-21.3393,55.4781,Le Battant Des Lames
398,Chaohu,CN,31.6000,117.8667,Chaohu Country Garden Phoenix Hotel
502,Palmas,BR,-10.2128,-48.3603,ibis Palmas
522,Correntina,BR,-13.3433,-44.6367,Correntina Palace Hotel


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df2.iterrows()]
locations = hotel_df2[["Latitude", "Longitude"]]

In [36]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='5px', wi…